In [1]:
import gensim 
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/zhouyou/Documents/PHD/summarization/resources/w2v_model/GoogleNews-vectors-negative300.bin', binary=True) 

In [2]:
from sklearn.cluster import AffinityPropagation
import os
import re
import sys
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer

In [3]:
document_path = "/Users/zhouyou/Downloads/doc/docsent"
corpus_add = "/Users/zhouyou/Downloads/doc/corpus_add"

In [4]:
def readfile(path):
    # 遍历文件夹
    files = os.listdir(path)
    files.sort()
    for file in files:
        raw_text = ''
        raw_text_list = []
        if not os.path.isdir(file):
            # print("File name: " + file)
            f = open(path + "/" + file)
            for line in f.readlines():
                searchObj = re.findall(r'SNO=(.*?)>(.*?)</S>', line)
                if searchObj:
                    # print(searchObj)
                    # print(searchObj[0][0])
                    if searchObj[0][0] == "\"1\"":
                        raw_text = raw_text + searchObj[0][1] + '.' + ' '
                        raw_text_list.append(searchObj[0][1])
                    else:
                        raw_text = raw_text + searchObj[0][1] + ' '
                        raw_text_list.append(searchObj[0][1])
                    # print(s)
            yield file, raw_text, raw_text_list

In [5]:
# 读原文的文件.
per_raw_text = readfile(document_path)

all_raw_text = []
all_filenames = []
all_raw_text_list = []

for filename, raw_text, raw_text_list in per_raw_text:
    all_filenames.append(filename)
    all_raw_text.append(raw_text)
    all_raw_text_list.append(raw_text_list)

# print(all_raw_text[0])
# print(len(all_raw_text))

tokenizer = RegexpTokenizer(r'\w+')

en_stop = get_stop_words('en')

# p_stemmer = PorterStemmer()
# p_stemmer = SnowballStemmer('english')

# wordnet 提取词干效果更好
p_stemmer = WordNetLemmatizer()

# texts 才是最后要放入dic的list

# 目标文本加入corpus
texts = []
for i in all_raw_text:
    tokens = tokenizer.tokenize(i)
    stopped_tokens = [i for i in tokens if not i in en_stop]
    # stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    stemmed_tokens = [p_stemmer.lemmatize(i) for i in stopped_tokens]
    # print(stemmed_tokens)
    texts.append(stemmed_tokens)
    break

In [6]:
print(all_raw_text[0])

 Chief Executive to renew ties with our trading partners . The Chief Executive , Mr Tung Chee Hwa , will visit Malaysia , Singapore , the United States , Japan , Belgium and the UK in the next two months to renew ties with our trading partners and update them on developments in Hong Kong after the handover . During these visits , Mr Tung will call on political and business leaders , and brief them on the smooth transition in Hong Kong to demonstrate our pride to return to China and our confidence in  "Hong Kong people ruling Hong Kong " under the  "one country , two systems " concept . Mr Tung will be in Kuala Lumpur on September 3 and call on Prime Minister Dr Mahathir Mohamad . He will be in Singapore on September 4 and 5 and call on Prime Minister Goh Chok Tong . Mr Tung will be in the United States from September 9 to 11 . He will visit Washington and New York . In Washington , Mr Tung looks forward to meeting with President Clinton . Mr Tung will also officiate at the opening cere

In [6]:
# # 随机文本加入corpus
# corpus_add_files = os.listdir(corpus_add)
# firstFile = False

# for file in corpus_add_files:
#     try:
#         corpus_add_file = open(corpus_add + "/" + file)
#         # print(corpus_add_file)
#         for line in corpus_add_file.readlines():
#             # print(line)
#             tokens = tokenizer.tokenize(line)
#             stopped_tokens = [i for i in tokens if not i in en_stop]
#             stemmed_tokens = [p_stemmer.lemmatize(i) for i in stopped_tokens]

#         texts.append(stemmed_tokens)
#         # print(len(stemmed_tokens))
#         # break
#     except:
#         pass

In [7]:
import csv
csvFile = open("/Users/zhouyou/Downloads/all-the-news/articles1.csv", "r")
reader = csv.reader(csvFile)
csv.field_size_limit(sys.maxsize)

131072

In [8]:
raw_text_corpus = []
for item in reader:
    if reader.line_num == 1:
        continue
    else:
        if reader.line_num > 2000:
            break
        # print(item[9])
        line = item[9]
        raw_text_corpus.append(line)
        tokens = tokenizer.tokenize(line)
        stopped_tokens = [i for i in tokens if not i in en_stop]
        stemmed_tokens = [p_stemmer.lemmatize(i) for i in stopped_tokens]

    texts.append(stemmed_tokens)

    

In [9]:
print(len(texts))

2000


# tf-idf

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [11]:
raw_text_corpus.append(all_raw_text[0])
print(len(raw_text_corpus))

2000


In [12]:
vectorizer = CountVectorizer()
transformer = TfidfTransformer()
word_frequence = vectorizer.fit_transform(raw_text_corpus)
words = vectorizer.get_feature_names()
tfidf = transformer.fit_transform(word_frequence)
# print(word_frequence[99])
# print(word_frequence[99].shape)
# tfidf 就是每篇文章的表示
print(tfidf.toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [13]:
# AP算法计算聚类中心数
ap = AffinityPropagation(preference=-50).fit(tfidf.toarray())
cluster_centers_indices = ap.cluster_centers_indices_
labels = ap.labels_

n_clusters_ = len(cluster_centers_indices)

print(n_clusters_)

2000


# WMD

# word embedding

In [ ]:
texttf = {}
for sent in texts:
    for i in sent:
#         print(i)
        if i in texttf:
            texttf[i] = texttf[i] + 1
        else:
            texttf[i] = 1
            
# print(len(texttf))

AP_input = []            
for i in texttf.keys():
    try:
        wordVec = model[i].tolist()
        wordVec.append(texttf[i])
        AP_input.append(wordVec)
    except:
        pass

print(len(AP_input))
    

In [11]:
# AP算法计算聚类中心数
ap = AffinityPropagation(preference=-50).fit(AP_input)
cluster_centers_indices = ap.cluster_centers_indices_
labels = ap.labels_

n_clusters_ = len(cluster_centers_indices)

print(n_clusters_)

103
